## Import Library dan Load Model

In [ ]:
import os
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from datasets import Dataset
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.optimizers import Adam

# Mengatur kebijakan mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

# Tentukan device untuk TensorFlow dan atur memory growth
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except RuntimeError as e:
        print(e)

# Load IndoBERT tokenizer dan model (TensorFlow version)
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
bert_model = TFBertModel.from_pretrained("indobenchmark/indobert-base-p1")


## Fungsi Membaca dan Membersihkan Data

In [ ]:
def read_text_files(folder_path):
    texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                content = file.read().strip().lower()
                if content:  # Check if the file is not empty
                    cleaned_content = clean_text(content)
                    texts.append(cleaned_content)
    return texts

def clean_text(text):
    unwanted_chars = ['*', '#', '_', ')', '(', '!', '?', '.', ',', '-']
    for char in unwanted_chars:
        text = text.replace(char, '')
    return text

# Membaca teks dari folder dataset
raw_texts = read_text_files('../Dataset/nlp_dataset')


## Tokenisasi dan Pembentukan Dataset

In [ ]:
# Fungsi untuk tokenisasi
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Membaca teks dari folder dataset
raw_texts = read_text_files('../Dataset/nlp_dataset')

# Membagi teks menjadi batch kecil sebelum tokenisasi
num_batches = 500  # Jumlah batch yang lebih banyak untuk mengurangi ukuran masing-masing batch
batch_size = len(raw_texts) // num_batches

# Membuat direktori untuk menyimpan subset dataset yang telah di-tokenize
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size if (i + 1) * batch_size < len(raw_texts) else len(raw_texts)
    batch_texts = raw_texts[start_idx:end_idx]

    # Tokenisasi batch kecil
    batch_dataset = Dataset.from_dict({"text": batch_texts})
    tokenized_batch = batch_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

    # Simpan tokenized batch ke disk
    tokenized_batch.save_to_disk(f'../saved_model/nlp_saved/nlp_01/tokenized/tokenized_dataset_batch_{i}')



## Membagi Dataset menjadi Beberapa Batch

In [ ]:
import os
import numpy as np
from datasets import load_from_disk, concatenate_datasets

# Inisialisasi variabel untuk menyimpan semua dataset tokenized
tokenized_datasets = []

# Looping untuk memuat dataset dari folder yang telah di-tokenisasi
for i in range(500):  # Sesuaikan dengan jumlah batch yang sudah Anda buat sebelumnya
    path = f'../saved_model/nlp_saved/nlp_01/tokenized_dataset_batch_{i}'
    if os.path.exists(path):
        tokenized_batch = load_from_disk(path)
        tokenized_datasets.append(tokenized_batch)
    else:
        print(f"Path {path} tidak ditemukan.")

# Menggabungkan semua dataset menjadi satu
if tokenized_datasets:
    merged_dataset = concatenate_datasets(tokenized_datasets)

    # Membagi dataset yang telah digabung menjadi batch yang lebih kecil
    num_batches = 500  # Meningkatkan jumlah batch menjadi 500 untuk ukuran batch yang sangat kecil
    batch_size = len(merged_dataset) // num_batches

    # Membuat direktori untuk menyimpan subset dataset yang baru
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size if (i + 1) * batch_size < len(merged_dataset) else len(merged_dataset)
        subset = merged_dataset.select(range(start_idx, end_idx))
        subset.save_to_disk(f'../saved_model/nlp_saved/nlp_01/new_tokenized/new_tokenized_dataset_batch_{i}')
else:
    print("Tidak ada dataset yang berhasil dimuat.")


## Definisi Model Custom untuk Sentence Embeddings

In [ ]:
class BertAutoencoder(tf.keras.Model):
    def __init__(self, bert_model):
        super(BertAutoencoder, self).__init__()
        self.bert = bert_model
        self.dense = tf.keras.layers.Dense(bert_model.config.vocab_size, activation='softmax', dtype='float32')

    def call(self, inputs):
        outputs = self.bert(**inputs)
        sequence_output = outputs.last_hidden_state
        # Rekonstruksi input dari embeddings yang dihasilkan
        reconstructed = self.dense(sequence_output)
        return reconstructed

# Instansiasi model autoencoder
autoencoder_model = BertAutoencoder(bert_model)


## Memuat Dataset dan Fine-Tuning Model

In [ ]:
from tensorflow.keras.mixed_precision import LossScaleOptimizer
from datasets import load_from_disk

# Optimizer menggunakan LossScaleOptimizer dengan dynamic loss scale
base_optimizer = Adam(learning_rate=2e-5)
optimizer = LossScaleOptimizer(base_optimizer)

# Compile model
autoencoder_model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Melatih model secara bertahap pada setiap batch kecil
num_batches = 500  # Jumlah batch yang lebih banyak
for i in range(num_batches):
    print(f"Training on batch {i+1}/{num_batches}")
    batch_dataset = load_from_disk(f'../saved_model/nlp_saved/nlp_01/new_tokenized/tokenized_dataset_batch_{i}')
    
    # Persiapan dataset untuk training
    train_dataset = batch_dataset.to_tf_dataset(
        columns=["input_ids", "attention_mask"],
        label_cols=["input_ids"],
        shuffle=True,
        batch_size=2,  # Mengurangi batch size menjadi 2
        collate_fn=None,
    )

    # Latih model
    autoencoder_model.fit(train_dataset, epochs=3)

# Simpan model setelah semua batch dilatih
autoencoder_model.save_weights('../saved_model/nlp_saved/nlp_01/sentence_embedding_model_incremental')
